In [1]:
import math
import numpy as np

In [4]:
def reynolds(rho:float,vel:float,l:float,mu:float) -> float :
    """
    Calcula el numero de Reynolds:
    
    Datos de entrada :
    - Densidad : rho [=] kg/m³
    - Velocidad : vel [=] m/s
    - Longitud dependiente del cambio de movimiento : l [=] m
    - Viscosidad del fludo : mu [=] kg/(m.s)
    
    Datos de salida :
    - Número de Reynolds : Re [=] adimensional
    """
    Re = (rho*vel*l)/(mu)
    return Re


def h_perdida_friccion(l : float,D: float,vel: float,f_d: float,g: float=9.8) -> float : 
    """
    Calcula la distancia que no alcanzo a recorrer por la fricción
    
    Datos de entrada:
    - longitud tubería : l [=] m
    - diametro de tubería : D [=] m
    - velocidad de flujo : vel [=] m/s
    - factor Darcy : f_d [=] m/s
    - gravedad : g [=] m/s², = 9.8
    
    Datos de salida:
    - Distancia que no pudo alcanzar por la fricción : h [=] m
    """
    
    h_f = (l*(vel**2)*f_d)/(2*D*g)
    
    return h_f


def f_darcy_laminar(Re: float) -> dict :
    """
    Calcula el factor de fricción de Darcy para flujo laminar.

    Datos de entrada:
    - Re : Número de Reynolds (adimensional). Válido en régimen turbulento (Re > 4000).

    Retorna
    - f : Factor de fricción de Darcy (adimensional).
    """
    
    if Re >= 2000:
        return {
            "message" : "Reynolds > 2000, flujo no laminar",
            "Factor Darcy" : f"este cálculo no aplica para Reynolds = {Re}"
        }
            
    f = 64/Re
    
    return {
        "message" : "Reynolds > 4000, flujo turbulento",
        "Factor Darcy" : f
    }


def f_darcy_shacham(Re: float, epsilon: float, D: float) -> dict :
    """
    Calcula el factor de fricción de Darcy usando la correlación explícita de Shacham.

    Datos de entrada:
    - Re : Número de Reynolds (adimensional). Válido en régimen turbulento (Re > 4000).
    - epsilon : Rugosidad absoluta de la tubería [=]m.
    - D : Diámetro interno de la tubería [=] m.

    Retorna
    - f : Factor de fricción de Darcy (adimensional).
    """

    if Re <= 4000:
        if Re < 2000 :
            return {
                "message" : "Reynolds es < 2000, se asume flujo laminar",
                "Factor Darcy" : f_darcy_laminar(Re)
            }
        else :
            return {
                "message" : "Reynolds no es > 4000, Reynolds no es < 2000, esta en transición entre laminar y turbulento",
                "Factor Darcy" : f"buscar modelos de calculo de f darcy para Re = {Re}"
            }

    rel_roughness = epsilon / D

    term1 = rel_roughness / 3.7
    term2 = 5.02 / Re
    inner_log = term1 + (14.5 / Re)

    argument = term1 - term2 * math.log10(inner_log)

    f = (-2.0 * math.log10(argument)) ** -2

    return {
        "message" : "Reynolds > 4000, flujo turbulento",
        "Factor Darcy" : f
    }


def h_perdidas_2k(K_1 : float, K_inf : float, D_i : float, Re : float, vel:float, g : float = 9.8) -> float :
    """
    Calcula las perdidas de energía en accesorios (Hooper).

    Datos de entrada:
    - K_1 : Tomado de datos documentados y manuales
    - K_inf : Tomado de datos documentados y manuales
    - D_i : Diámetro interno de la tubería [=] m.
    - Re : Número de Reynolds [=] adimensional.
    - vel : velocidad del fluido que pasa por el accesorio [=] m/s.
    - g : gravedad [=] m/s, default = 9.8.

    Retorna
    - f : Factor de fricción de Darcy (adimensional).
    """
    D_i = D_i * 39.3701 # inches
    K = (K_1/Re) + (K_inf * (1 + (1/D_i)))
    h_perdida = K * ((vel**2)/2*g)
    return h_perdida


def h_perdidas_3k(K_1 : float, K_i : float, K_d : float, D_i : float, Re : float, vel:float, g : float = 9.8) -> float :
    """
    Calcula las perdidas de energía en accesorios (Darby).

    Datos de entrada:
    - K_1 : Tomado de datos documentados y manuales
    - K_i : Tomado de datos documentados y manuales
    - K_d : Tomado de datos documentados y manuales
    - D_i : Diámetro interno de la tubería [=] m.
    - Re : Número de Reynolds [=] adimensional.
    - vel : velocidad del fluido que pasa por el accesorio [=] m/s.
    - g : gravedad [=] m/s, default = 9.8.

    Retorna
    - f : Factor de fricción de Darcy (adimensional).
    """
    D_i = D_i * 39.3701 # inches
    K = (K_1/Re) + (K_i * (1 + (K_d/(D_i**0.3))))
    h_perdida = K * ((vel**2)/2*g)
    return h_perdida